In [ ]:
#Need to run when data loading for the first time
!pip install opensearch-py

In [ ]:
index =  ""
language = ""
endpoint_name = ""
image_endpoint_name = ''

In [ ]:
import json
from opensearch_search import add_products
from embeddings import get_embedding_sagemaker,get_image_embedding_sagemaker
import csv
local_file = "../docs/algolia_offers1000.csv"

product_info_list = []
product_embedding_list = []
image_embedding_list = []
metadatas = []
i = 0
image_name = 'mainImage'

csvfile=open(local_file,mode='r',encoding='utf-8')
reader = [each for each in csv.DictReader(csvfile, delimiter=',')]

for line in reader:
    metadata = {}
    image_url = ''
    for key in line.keys():
        metadata[key.replace('\ufeff','').strip()] = line[key].strip()
        if key == image_name:
            image_url = line[key].strip()

    image_embedding = ''
    if len(image_url) > 0:
        try:
            image_embedding = get_image_embedding_sagemaker(image_endpoint_name,image_url)
        except:
            print("image embedding error at:" + str(i))
    else:
        print('image_url is none')
        break

    if len(image_embedding) > 0:
        image_embedding_list.append(image_embedding)
        metadatas.append(metadata)
        product_info_list.append(image_url)

    i += 1
    if i % 100 == 0:
        print('data number:',i)
    if i % 100 == 0 and len(image_embedding_list) > 0:
        add_products(index,product_info_list,product_embedding_list,metadatas,image_embedding_list)
        product_info_list = []
        product_embedding_list = []
        image_embedding_list = []
        metadatas = []

print('total data number:',i)

add_products(index,product_info_list,product_embedding_list,metadatas,image_embedding_list)

print('finish add products to opensearch,index is:' + index)